# Classification

## Classification Service

In [1]:
import pandas as pd
import numpy as np

In [3]:
data_source = "https://stats.idre.ucla.edu/stat/data/binary.csv"
df = pd.read_csv(data_source)

df[:5]

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [7]:
y_data = df["admit"].values.reshape(-1,1)
x_data = df.iloc[:,1:].values

y_data[:5], x_data[:5]

(array([[0],
        [1],
        [1],
        [1],
        [0]], dtype=int64),
 array([[380.  ,   3.61,   3.  ],
        [660.  ,   3.67,   3.  ],
        [800.  ,   4.  ,   1.  ],
        [640.  ,   3.19,   4.  ],
        [520.  ,   2.93,   4.  ]]))

In [8]:
from sklearn import preprocessing # Min-Max Standardzation

min_max_scaler = preprocessing.MinMaxScaler()
x_data = min_max_scaler.fit_transform(x_data)

x_data[:5]

array([[0.27586207, 0.77586207, 0.66666667],
       [0.75862069, 0.81034483, 0.66666667],
       [1.        , 1.        , 0.        ],
       [0.72413793, 0.53448276, 1.        ],
       [0.51724138, 0.38505747, 1.        ]])

In [9]:
from sklearn import linear_model, datasets

logreg = linear_model.LogisticRegression(fit_intercept=True)
logreg.fit(x_data, y_data.ravel())

sum(logreg.predict(x_data) == y_data.ravel()) / len(y_data.ravel())

0.705

In [10]:
theta = np.append(logreg.intercept_, logreg.coef_.ravel())
theta

array([-1.50619015,  1.06983642,  1.11573376, -1.47695382])

In [11]:
theta_file_name = "theta_bin.npy"
np.save(theta_file_name, theta)    

In [12]:
np.load(theta_file_name)

array([-1.50619015,  1.06983642,  1.11573376, -1.47695382])

In [14]:
min_max = np.vstack( (df.iloc[:,1:].values.min(axis=0), df.iloc[:,1:].values.max(axis=0)) )
min_max

array([[220.  ,   2.26,   1.  ],
       [800.  ,   4.  ,   4.  ]])

In [15]:
min_max_file_name = "min_max.npy"
np.save(min_max_file_name, min_max)

In [17]:
np.load(min_max_file_name)

array([[220.  ,   2.26,   1.  ],
       [800.  ,   4.  ,   4.  ]])

In [18]:
np.load(theta_file_name)

array([-1.50619015,  1.06983642,  1.11573376, -1.47695382])

In [19]:
import pickle

f = open('logicmodel.pkl', 'wb')
f.seek(0)

pickle.dump(logreg,f )
f.close()

In [20]:
fr = open('./logicmodel.pkl', 'rb')
logreg_pickle = pickle.load(fr)

sum(logreg.predict(x_data) == logreg_pickle.predict(x_data)) / len(y_data.ravel())


1.0

## Logistic_service_server

In [ ]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import time
import cgi
import numpy as np


hostName = "localhost"
hostPort = 9000

class StoreHandler(BaseHTTPRequestHandler):
    def do_POST(self):

        form = cgi.FieldStorage(
            fp=self.rfile,
            headers=self.headers,
            environ={'REQUEST_METHOD':'POST',
                     'CONTENT_TYPE':self.headers['Content-Type'],
                     })
        gre_score = float(form['gre'].value)
        gpa_score = form['gpa'].value
        rank_score = form['rank'].value

        min_max_file = "min_max.npy"
        theta_file = "theta_bin.npy"

        theta = np.load(theta_file)
        min_max = np.load(min_max_file)
        x_data = np.array([gre_score,gpa_score, rank_score], dtype=np.float32)

        x_data_rescale = (x_data - min_max[0]) / (min_max[1] - min_max[0])
        x_data_rescale = np.insert(x_data_rescale, 0, 1)

        def sigmoid(z):
            return 1 / (1 + np.exp(z))

        def hypothesis_function(x, theta):
            z = (np.dot(-x, theta))
            return sigmoid(z)

        h_result = hypothesis_function(x_data_rescale, theta)
        print(h_result)
        print(theta)

        result_set = h_result > 0.5

        if result_set == True:
            result = ("Get Admission!!")
        else:
            result = ("Fail")

        self.respond(result.encode())

    def do_GET(self):
        response = b"""
<!DOCTYPE html>
<html lang="en">
<body>
    <p>Check your admission status!</p>
    <form method="post">
        <p>gre: <input type="text" name="gre"></p>
        <p>gpa: <input type="text" name="gpa"></p>
        <p>rank: <input type="text" name="rank"></p>
        <p><input type="submit" value="submit"></p>
    </form>
</body>
</html>
        """

        self.respond(response)

    def respond(self, response, status=200):
        self.send_response(status)
        self.send_header("Content-type", "text/html")
        self.send_header("Content-length", len(response))
        self.end_headers()
        self.wfile.write(response)


myServer = HTTPServer((hostName, hostPort), StoreHandler)
print(time.asctime(), "Server Starts - %s:%s"    % (hostName, hostPort))

try:
    myServer.serve_forever()
except KeyboardInterrupt:
    pass

myServer.server_close()
print(time.asctime(), "Server Stops - %s:%s" % (hostName, hostPort))

Mon Feb  8 16:16:31 2021 Server Starts - localhost:9000


127.0.0.1 - - [08/Feb/2021 16:19:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2021 16:19:39] "GET /favicon.ico HTTP/1.1" 200 -
